# Try scraping model cards

In [ ]:
import pandas as pd
import numpy as np
import requests
from huggingface_hub import HfApi, snapshot_download, hf_hub_download, ModelCard
from huggingface_hub.utils import http_backoff
from typing import Dict, Any
import backoff

HF_TOKEN = userdata.get('HF-TOKEN')

!huggingface-cli login --token $HF_TOKEN

In [ ]:
model_dataset_df = pd.read_csv("model_dataset_combined_jul12.csv")
model_dataset_df.head()

In [ ]:
import os
import sys
import json
import signal
import requests
import backoff
import threading
import time
from tqdm import tqdm
from typing import List, Dict, Any, Generator
from huggingface_hub import HfApi, ModelCard
import re

import csv

from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import requests
import threading

api = HfApi()
model_id = "apple/DiffuCoder-7B-cpGRPO"
model_dataset_df['model_card'] = ""
model_dataset_df['metadata'] = ""


# Define the CSV file path
csv_file_path = "2m_models_with_cards_and_metadata_jul12.csv"

# Function to write a row to the CSV file
def write_to_csv(index, model_id, fulljson, metadata, model_card):
    with threading.Lock():
        with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([index, model_id, fulljson, metadata, model_card])

# Parallelize the fetching of metadata and model cards using ThreadPoolExecutor
def fetch_metadata_and_model_card(index):
    delay = 0.1  # Start with 500ms
    max_retries = 5

    model_id = model_dataset_df.iloc[index]["modelId"]

    # Fetch metadata with exponential backoff
    for _ in range(max_retries):
        try:
            metadata = api.model_info(model_id).__dict__
            model_dataset_df.at[index, "metadata"] = metadata
            break
        except requests.exceptions.RequestException as e:
            if "404" in str(e):
                print(f"Model {model_id} not found, skipping...")
                metadata = None
                break
            else:
                print(f"Error fetching metadata: {e}, backing off for {delay:.2f}s...")
                time.sleep(delay)
                delay *= 2
    else:
        print(f"Failed to fetch metadata for model {model_id} after {max_retries} retries.")
        metadata = None

    # Wait for 1/5 of a second before fetching the model card
    time.sleep(0.2)

    # Fetch model card with exponential backoff
    delay = 0.5  # Reset delay for model card fetching
    for _ in range(max_retries):
        try:
            model_card = ModelCard.load(model_id)
            model_dataset_df.at[index, "model_card"] = model_card
            break
        except requests.exceptions.RequestException as e:
            if "404" in str(e):
                print(f"Model {model_id} not found, skipping...")
                model_card = None
                break
            else:
                print(f"Error fetching model card: {e}, backing off for {delay:.2f}s...")
                time.sleep(delay)
                delay *= 2
    else:
        print(f"Failed to fetch model card for model {model_id} after {max_retries} retries.")
        model_card = None

    # Write the results to the CSV file
    write_to_csv(index, model_id, model_dataset_df.at[index, "fullJson"], metadata, model_card)

# Initialize the CSV file with headers
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["index","modelId","fullJson", "Metadata", "ModelCard"])

# Use ThreadPoolExecutor to parallelize the process
MAX_WORKERS_THREAD = 20

with ThreadPoolExecutor(max_workers=MAX_WORKERS_THREAD) as executor:
    futures = [executor.submit(fetch_metadata_and_model_card, i) for i in range(len(model_dataset_df))]
    for i, future in enumerate(as_completed(futures), 1):
        try:
            future.result()  # To raise any exceptions caught during execution
        except Exception as e:
            print(f"Error processing future {i}: {e}")
        if i % 1000 == 0:
            print(f"Completed {i}/{len(futures)} tasks")
            print()
            print()
            print()
    for future in as_completed(futures):
        future.result()  # To raise any exceptions caught during execution
